In [ ]:
### Import libraries
from netCDF4 import Dataset
import xarray as xr
import numpy as np
import cartopy.crs as crs
import matplotlib
from matplotlib.cm import get_cmap
import matplotlib.pyplot as plt
from wrf import getvar, destagger, ll_to_xy
import warnings
from scipy.interpolate import griddata
from matplotlib.cm import get_cmap
import cartopy.crs as crs
from cartopy.feature import NaturalEarthFeature
warnings.filterwarnings('ignore')
states = NaturalEarthFeature(category="cultural", scale="50m",
                             facecolor="none",
                             name="admin_1_states_provinces")

nx = 3000
ny = 1875
nz = 59
dx = 4000
G = 9.81
a = 0.1
h = 0.1
l = 32000


In [ ]:
### Compute the perturbations
rst_dir = '/p/work1/lloveras/real/jan2022/wrf/4km_files/wave/wrfrst_d01_2022-01-27_00_00_00'
ds = xr.open_dataset(rst_dir)

xg = np.zeros((ny,nx))
for i in range(int(nx/2),nx):
    xg[:,i] = xg[:,i-1] + dx/ds['MAPFAC_MX'][0,:,i]

for i in range(int(nx/2)-1,-1,-1):
    xg[:,i] = xg[:,i+1] - dx/ds['MAPFAC_MX'][0,:,i]
    
xg = xg - np.amin(xg)

yg = np.zeros((ny,nx))
for j in range(int(ny/2),ny):
    yg[j,:] = yg[j-1,:] + dx/ds['MAPFAC_MY'][0,j,:]

for j in range(int(ny/2)-1,-1,-1):
    yg[j,:] = yg[j+1,:] - dx/ds['MAPFAC_MY'][0,j,:]
    
yg = yg - np.amin(yg)

z = np.asarray(ds['ZNU'][0])

pert = np.zeros((nz,ny,nx))
for k in range(nz):
    pert[k,:,:] = a*np.exp(-z[k]/h)*np.sin(2*np.pi*xg/l)*np.sin(2*np.pi*yg/l)

pert = np.flip(pert, axis=0)


In [ ]:
### Add the perturbations
ncfile = Dataset(rst_dir,'r+')
th1 = np.asarray(ncfile.variables['THM_1'][0,:,:,:])
th2 = np.asarray(ncfile.variables['THM_2'][0,:,:,:])
ncfile.variables['THM_1'][0,:,:,:] = th1 + pert
ncfile.variables['THM_2'][0,:,:,:] = th2 + pert
ncfile.close()
